# Quadp
## GUI frontend for program
## Manages realsense cam vision and exporting to .ply file
---
#### Creation Date 9/7/22
#### Author: Dalton Bailey
#### Course: CSCI 490
#### Instructor Sam Siewert
----
- **Planning Pothole potential using pointcloud from a ply file, powered by python, pacing on the pipad**
- Notebook primarily useful to explain individual code blocks, code functions best when ran using _python3 quadp.py_ or using the script: _./quadp_ 


### Required Imports 
----
- yaml: Allows program to save configuration data to yaml file 
- tkinter: Creates GUI and embedded gui widgets
- numpy: Performs complex calculations
- cv2: Handles piping camera vision to GUI
- atexit: Performs functionality on program exit
- os: Allows program to run os commans
- themes: Custom library for holding themes
- calibration: Calibrates Realsense Camera
- calc: Calculation backend (see calc.ipynb)
- pyrealsense2: Python Realsense library (allows control of Realsense Camera)
- time: Used to time code blocks
- PIL: Python Image Library, handles image conversion and processing
- webview: Opens embedded webbrowser for Documentation

In [1]:
import yaml
import tkinter as tk
from tkinter import ttk
import numpy as np
import cv2 as cv2
import matplotlib.pyplot as plt
import atexit
import os
from os.path import exists
from themes import *
import calibration as cal
from calc import *
import pyrealsense2 as rs
import time
import PIL as pil
from PIL import ImageTk
from IPython.display import clear_output  # Clear the screen
import webview

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


#### Class variables for gui
----

In [2]:
class interface():
    # Class variables (Initialize all as none until they are required)
    root = None
    working_dir = None
    output_file = None
    scanning = None

    # User config variables
    conf_file = None
    conf = None
    username = None
    debug = None
    units = None
    density = None
    densityUnit = None

    # Global GUI Variables
    theme = None
    screen_width = None
    screen_height = None

    # Global Tkinter Widgets
    video_out = None
    cam_controls = None
    s_scan_button = None
    export_button = None

#### GUI Constructor 
----
- Determines width of screen 
- Sets global Tkinter widgets 

In [3]:
    # Constructor for Interface object
    def __init__(self):
        self.root = tk.Tk()  # Calls tktinker object and sets self.root to be equal to it
        self.calcBackend = pholeCalc()  # Initialize the calculation backend
        self.working_dir = os.path.dirname(os.path.realpath(__file__))
        #self.working_dir = os.getcwd()
        self.conf_file = self.working_dir+'/data/conf.yml'

        # Generate unique hash to store export of scan (takes some time)
        self.output_file = self.working_dir + "/data/ply/" + self.calcBackend.hash(
            (''.join(random.choice(string.ascii_letters) for i in range(7)))) + ".ply"

        # Load configuration from yaml file
        self.loadConfig()

        self.screen_width = self.root.winfo_screenwidth()  # Get width of current screen
        self.screen_height = self.root.winfo_screenheight()  # Get height of current screen

        # Set program title and geometry of root gui
        self.root.title("Quad-P")
        self.root.geometry("%dx%d" % (self.screen_width, self.screen_height))

        # Configure GUI title and Geometry
        self.root.configure(background=themes[self.theme]['background_colo'])

        # Create Location for video feed in GUI
        self.video_out = tk.Canvas(
            self.root, bg="#000000", height=480, width=640, borderwidth=5, relief="sunken")
        self.video_out.grid(column=0, row=1, columnspan=10,
                            pady=35, ipadx=5, ipady=5, sticky=tk.NS)
        # Create Location for text output in GUI
        self.cam_controls = tk.Label(self.root, fg=themes[self.theme]['background_colo'], bg=themes[self.theme]['background_colo'], height=round(
            self.screen_height*0.002555), width=round(self.screen_width*0.059))
        self.cam_controls.grid(column=0, row=2, columnspan=10,
                               sticky=tk.NS)

        self.s_scan_button = tk.Button(
            self.cam_controls, text="Enable Camera", command=lambda: self.startScan())
        self.s_scan_button.grid(column=1, row=0, padx=20)

        self.export_button = tk.Button(
            self.cam_controls, text="Export Scan", command=lambda: self.exportScan())
        self.export_button.grid(column=3, row=0, padx=20)

        # Create Location for text output in GUI
        self.b_data = tk.Label(self.root, fg=themes[self.theme]['main_colo'], bg=themes[self.theme]['main_colo'], height=round(
            self.screen_height*0.00555), width=round(self.screen_width*0.059), borderwidth=5, relief="solid")
        self.b_data.grid(column=0, row=3, columnspan=10,
                         sticky=tk.SW)

#### Livestream Camera's Vision
---

In [ ]:
    def startScan(self):
        pipe = rs.pipeline()                      # Create a pipeline
        cfg = rs.config()                         # Create a default configuration
        print("[QUAD_P] Pipeline is created") if gui.debug else None

        print("[QUAD_P] Searching For Realsense Devices..") if gui.debug else None
        selected_devices = []                     # Store connected device(s)

        for d in rs.context().devices:
            selected_devices.append(d)
            print(d.get_info(rs.camera_info.name))
        if not selected_devices:
            print("No RealSense device is connected!")
            return

        print(
            "[QUAD_P] (debug) Streaming camera vision to GUI... ") if gui.debug else None

        rgb_sensor = depth_sensor = None

        for device in selected_devices:
            print("Required sensors for device:",
                  device.get_info(rs.camera_info.name))
            for s in device.sensors:                              # Show available sensors in each device
                if s.get_info(rs.camera_info.name) == 'RGB Camera':
                    print("[QUAD_P] - RGB sensor found") if gui.debug else None
                    rgb_sensor = s                                # Set RGB sensor
                if s.get_info(rs.camera_info.name) == 'Stereo Module':
                    depth_sensor = s                              # Set Depth sensor
                    print("[QUAD_P] - Depth sensor found") if gui.debug else None
        # Mapping depth data into RGB color space
        colorizer = rs.colorizer()
        # Configure and start the pipeline
        profile = pipe.start(cfg)

        # Show 1 row with 2 columns for Depth and RGB frames
        fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(24, 8))
        # Title for each frame
        title = ["Depth Image", "RGB Image"]

        # Skip first frames to give syncer and auto-exposure time to adjust
        for _ in range(10):
            frameset = pipe.wait_for_frames()

        # Increase to display more frames
        for _ in range(30):
            # Read frames from the file, packaged as a frameset
            frameset = pipe.wait_for_frames()
            depth_frame = frameset.get_depth_frame()              # Get depth frame
            color_frame = frameset.get_color_frame()              # Get RGB frame

            # This is what we'll actually display
            colorized_streams = []
            if depth_frame:
                colorized_streams.append(np.asanyarray(
                    colorizer.colorize(depth_frame).get_data()))
            if color_frame:
                colorized_streams.append(np.asanyarray(color_frame.get_data()))

            # Iterate over all (Depth and RGB) colorized frames
            for i, ax in enumerate(axs.flatten()):
                if i >= len(colorized_streams):
                    continue          # When getting less frames than expected
                # Set the current Axes and Figure
                plt.sca(ax)
                # colorized frame to display
                plt.imshow(colorized_streams[i])
                # Add title for each subplot
                plt.title(title[i])
            # Clear any previous frames from the display
            clear_output(wait=True)
            # Adjusts display size to fit frames
            plt.tight_layout()
            # Make the playback slower so it's noticeable
            plt.pause(1)

        pipe.stop()                                               # Stop the pipeline
        print("[QUAD_P] Done!")

#### Stop Livestreaming Camera Vision 
----

In [ ]:
    def stopScan(self):
        print("[QUAD_P] (debug) Disabling live feed...") if gui.debug else None
        self.scanning = False
        self.s_scan_button = tk.Button(
            self.cam_controls, text="Enable Camera", command=lambda: self.stopScan())
        self.s_scan_button.grid(column=1, row=0, padx=20)
        if self.export_scan:
            self.exportScan()

#### Export Scan into a .ply file
----

In [ ]:
    def exportScan(self):
        start_time = time.process_time()  # start timer
        print("Searching For Realsense Devices..")
        selected_devices = []                     # Store connected device(s)

        for d in rs.context().devices:
            selected_devices.append(d)
            print(d.get_info(rs.camera_info.name))
        if not selected_devices:
            print("No RealSense device is connected!")
            return

        print(
            "[QUAD_P] (debug) Exporting camera's vison as .ply file...") if gui.debug else None
        # Declare pointcloud object, for calculating pointclouds and texture mappings
        pc = rs.pointcloud()
        # We want the points object to be persistent so we can display the last cloud when a frame drops
        points = rs.points()

        # Declare RealSense pipeline, encapsulating the actual device and sensors
        pipe = rs.pipeline()
        config = rs.config()
        # Enable depth stream
        config.enable_stream(rs.stream.depth)

        # Start streaming with chosen configuration
        pipe.start(config)

        # We'll use the colorizer to generate texture for our PLY
        # (alternatively, texture can be obtained from color or infrared stream)
        colorizer = rs.colorizer()

        try:
            # Give camera time to adjust to exposure
            for x in range(10):
                pipe.wait_for_frames()

            # Wait for the next set of frames from the camera
            frames = pipe.wait_for_frames()
            colorized = colorizer.process(frames)

            # Create save_to_ply object
            ply = rs.save_to_ply(self.output_file)

            # Set options to the desired values
            # In this example we'll generate a textual PLY with normals (mesh is already created by default)
            ply.set_option(rs.save_to_ply.option_ply_binary, False)
            ply.set_option(rs.save_to_ply.option_ply_normals, True)

            print("[QUAD_P] (debug) Saving to ",
                  self.output_file, "...") if gui.debug else None

            # Apply the processing block to the frameset which contains the depth frame and the texture
            ply.process(colorized)

            print(f"[QUAD_P] (debug) Export Complete!\n Elapsed time was ",
                  (time.process_time() - start_time) * 1000, "ms.\n") if gui.debug else None
        finally:
            pipe.stop()